In [1]:
import numpy as np
import scipy.sparse
from scipy.sparse.linalg import lobpcg, eigsh, minres, LinearOperator
from scipy.sparse import csr_matrix
import time
from utils.tools import build_weighted_bethe_hessian
from scipy.sparse.csgraph import minimum_spanning_tree

In [1]:
from petsc4py import PETSc
from slepc4py import SLEPc

In [2]:
S = SLEPc.SVD()

In [ ]:
S.setLanczosOneSide

In [2]:
adj_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.adj"
adjacency_matrix = scipy.sparse.load_npz(adj_path + ".npz")
adjacency_matrix.data = adjacency_matrix.data ** 0.3
degrees = np.asarray(adjacency_matrix.sum(axis=1)).flatten()

In [3]:
r = np.sqrt(np.mean(degrees**2) / np.mean(degrees) - 1)
#r = np.mean(adjacency_matrix.data**2)
D, A = build_weighted_bethe_hessian(adjacency_matrix, r)

n = adjacency_matrix.shape[0]
I = scipy.sparse.eye(n, format='csr')
Hr = D - A #+ I * np.mean(degrees)

In [5]:
class MatrixOperator(object):

    def __init__(self, A):
        self.A = A.astype(PETSc.ScalarType)
        self.n_calls = 0

    def mult(self, A, x, y):
        xx = x.getArray(readonly=1)
        yy = y.getArray(readonly=0)
        yy[:] = self.A.dot(xx)
        self.n_calls += 1
        
    def getDiagonal(self, A, y):
        yy = y.getArray(readonly=0)
        yy[:] = self.A.diagonal()

In [17]:
from petsc4py import PETSc
from slepc4py import SLEPc

A = Hr
k = 100
tol = 1e-2
max_iter = 100

### Setup matrix operator
n = A.shape[0]
mat = MatrixOperator(A)
A_operator = PETSc.Mat().createPython([n, n], mat)
A_operator.setUp()

#A_operator = PETSc.Mat().createAIJ(size=A.shape, csr=(A.indptr, A.indices, A.data))

In [18]:
### Solve eigenproblem
E = SLEPc.EPS()
E.create()
E.setOperators(A_operator)
E.setProblemType(SLEPc.EPS.ProblemType.HEP)
E.setType(SLEPc.EPS.Type.LANCZOS)
E.setDimensions(k)
E.setTolerances(tol, max_iter)
E.setWhichEigenpairs(SLEPc.EPS.Which.SMALLEST_REAL)

def monitor_fun(eps, iters, nconv, eigs, errors):
    print("Current iteration: %d, number of converged eigenvalues: %d" % (iters, nconv))

E.setMonitor(monitor_fun)

print("Starting solving")
start = time.time()
E.solve()
print("Time_elapsed: %d" % (time.time() - start))
print("Number of calls to Ax: %d" % mat.n_calls)

Starting solving
Current iteration: 1, number of converged eigenvalues: 49
Current iteration: 2, number of converged eigenvalues: 76
Current iteration: 3, number of converged eigenvalues: 95
Current iteration: 4, number of converged eigenvalues: 101
Time_elapsed: 182
Number of calls to Ax: 686


In [10]:
### Collect results
print("")
its = E.getIterationNumber()
print("Number of iterations of the method: %i" % its)
sol_type = E.getType()
print("Solution method: %s" % sol_type)
nev, ncv, mpd = E.getDimensions()
print(nev, ncv, mpd)
print("Number of requested eigenvalues: %i" % nev)
tol, maxit = E.getTolerances()
print("Stopping condition: tol=%.4g, maxit=%d" % (tol, maxit))
nconv = E.getConverged()
print("Number of converged eigenpairs: %d" % nconv)
nconv = min(nconv, k)

if nconv < k:
    raise ZeroDivisionError("Failed to converge for requested number of k with maxiter=%d" % max_iter)

vecs = np.zeros([n, nconv])
vals = np.zeros(nconv)

xr, tmp = A_operator.getVecs()
xi, tmp = A_operator.getVecs()

if nconv > 0:
    for i in range(nconv):
        val = E.getEigenpair(i, xr, xi)
        vals[i] = val.real
        vecs[:, i] = xr


Number of iterations of the method: 4
Solution method: lanczos
100 200 200
Number of requested eigenvalues: 100
Stopping condition: tol=0.01, maxit=100
Number of converged eigenpairs: 101


In [11]:
vals

array([-52.81019524, -11.47311448,  -7.327142  ,  -5.68407192,
        -5.01466348,  -4.62261124,  -4.37962547,  -3.95044248,
        -3.50956361,  -3.314219  ,  -3.17637015,  -2.69915527,
        -2.5935662 ,  -2.35605041,  -2.32639613,  -2.00888757,
        -1.85249348,  -1.80078643,  -1.71902306,  -1.66647199,
        -1.53121169,  -1.45979545,  -1.39323969,  -1.35780827,
        -1.29056561,  -1.17121047,  -1.12774989,  -1.09565135,
        -1.06909956,  -1.01114   ,  -0.98329121,  -0.91351126,
        -0.8518678 ,   7.35872008,   7.45666826,   7.50411991,
         8.01808214,   8.07617109,   8.4343341 ,   8.46181108,
         8.53984821,   8.63057676,   8.73587394,   8.92982677,
         9.06431769,   9.19448371,   9.27837346,   9.43445328,
         9.64921205,   9.71147721,   9.81693597,   9.88986496,
         9.96266172,  10.09344502,  10.25243466,  10.41685042,
        10.46698245,  10.68468045,  10.75888908,  10.94919774,
        11.0848075 ,  11.34126278,  11.80338264,  12.17

In [18]:
output_path = "../data/wiki/win=1_correctly_weighted_bethe_hessian_fast_slepc_pow=0.3_dim=100"
np.save(output_path + ".vecs", vecs[:,:100])
np.save(output_path + ".vals", vals[:100])
np.save(output_path + ".degrees", degrees)

In [19]:
import shutil
base_path = '../data/wiki/'
shutil.copyfile(base_path + 'win=1_weighted_bethe_hessian_slepc_scaled_abs_tol=1e-3_pow=0.3_dim=500.words.vocab', 
                base_path + 'win=1_correctly_weighted_bethe_hessian_fast_slepc_pow=0.3_dim=100' + ".words.vocab")

'../data/wiki/win=1_correctly_weighted_bethe_hessian_fast_slepc_pow=0.3_dim=100.words.vocab'